#### 데이터 로드

In [6]:
val = pd.read_json('datas/val.json', typ = 'frame')
song_tags = pd.read_json('datas/song_tags.json', typ = 'frame')
tag_songs = pd.read_json('datas/tag_songs.json', typ = 'frame')

#### val의 song에서 tag 채우기 (1)

In [30]:
val = pd.read_json('datas/val.json', typ = 'frame')
val

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,[],45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,[],79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,[잔잔한],101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[어머니, 힘들때, 아빠, 가족, 위로받고싶을때]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,[],77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,[],36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [33]:
from collections import Counter
from tqdm import notebook
val_copy = val.copy()
# df_indx = i
for i in notebook.tqdm(range(0,len(val_copy))):
    tags = {}
    if len(val_copy['songs'][i]) != 0:
        # song_idx = j
        for j in range(0,len(val_copy['songs'][i])):
            if len(song_tags[song_tags['songs'] == val_copy['songs'][i][j]]) != 0:
                tag = song_tags[song_tags['songs'] == val_copy['songs'][i][j]]['tags_num'].values[0]
                tags = Counter(tags) + Counter(tag)
            val_copy['tags'][i] = list(dict(sorted(tags.items(), key = lambda x:x[1],reverse=True)[:10]).keys())

In [ ]:
# 저장
val_copy.to_json('datas/val_copy.json')

#### val_copy의 tag를 기반으로 song 붙이기 (tag에서 song)(1-2)

In [35]:
# 불러오기
val_copy = pd.read_json('datas/val_copy.json')

In [36]:
tag_songs_index = tag_songs.set_index(keys = 'tags')

In [38]:
# 일단 상위 100개만 가져오게
import itertools
sample = tag_songs[tag_songs['tags'] ==val_copy['tags'][0][0]]['songs_num'].values[0]
dict(itertools.islice(sample.items(),100))

{'654757': 227,
 '88503': 197,
 '645489': 191,
 '224921': 190,
 '492678': 177,
 '281838': 165,
 '269370': 164,
 '482903': 163,
 '295250': 155,
 '37748': 143,
 '246984': 143,
 '152475': 141,
 '123465': 138,
 '707724': 135,
 '140130': 133,
 '316600': 132,
 '88471': 130,
 '140833': 130,
 '118219': 127,
 '260220': 126,
 '6546': 123,
 '689725': 118,
 '701557': 117,
 '48948': 117,
 '211158': 115,
 '11657': 113,
 '556394': 112,
 '208186': 112,
 '264701': 111,
 '200877': 110,
 '35703': 106,
 '58773': 105,
 '321933': 104,
 '393654': 103,
 '592526': 102,
 '589983': 101,
 '424531': 99,
 '507358': 99,
 '376126': 99,
 '438929': 98,
 '375450': 95,
 '70641': 94,
 '456581': 94,
 '207558': 94,
 '106756': 92,
 '608260': 92,
 '544584': 91,
 '400781': 91,
 '366565': 90,
 '380832': 89,
 '321564': 88,
 '563956': 86,
 '272379': 85,
 '630629': 85,
 '266273': 85,
 '152422': 82,
 '385430': 82,
 '291805': 82,
 '6716': 81,
 '205757': 81,
 '264338': 79,
 '583279': 79,
 '682575': 77,
 '77017': 77,
 '531574': 77,
 '

In [39]:
from collections import Counter
from tqdm import notebook
val_copy2 = val_copy.copy()
# df_indx = i
for i in notebook.tqdm(range(0, len(val_copy2))):
    songs = {}
    if len(val_copy2['tags'][i]) != 0:
        # song_idx = j
        for j in range(0, len(val_copy2['tags'][i])):
            if len(tag_songs[tag_songs['tags'] == val_copy2['tags'][i][j]]) != 0:
                song = tag_songs[tag_songs['tags'] ==val_copy2['tags'][i][j]]['songs_num'].values[0]
                song = dict(itertools.islice(song.items(), 1000)) # 한 태그당 가져올 노래 갯수 지정
                songs = Counter(songs) + Counter(song)
                val_copy2['songs'][i] = list(
                    dict(sorted(songs.items(), key=lambda x: x[1], reverse=True)[:100]).keys())
    else:
        print(f'{i}번째 태그없음')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


1번째 태그없음
9번째 태그없음
35번째 태그없음
57번째 태그없음
60번째 태그없음
71번째 태그없음
101번째 태그없음
115번째 태그없음
135번째 태그없음
167번째 태그없음
199번째 태그없음
218번째 태그없음
220번째 태그없음
221번째 태그없음
233번째 태그없음
240번째 태그없음
253번째 태그없음
254번째 태그없음
257번째 태그없음
279번째 태그없음
300번째 태그없음
304번째 태그없음
307번째 태그없음
308번째 태그없음
317번째 태그없음
333번째 태그없음
334번째 태그없음
352번째 태그없음
379번째 태그없음
384번째 태그없음
391번째 태그없음
400번째 태그없음
416번째 태그없음
420번째 태그없음
430번째 태그없음
440번째 태그없음
454번째 태그없음
466번째 태그없음
474번째 태그없음
475번째 태그없음
481번째 태그없음
490번째 태그없음
498번째 태그없음
540번째 태그없음
547번째 태그없음
572번째 태그없음
598번째 태그없음
601번째 태그없음
613번째 태그없음
615번째 태그없음
633번째 태그없음
648번째 태그없음
675번째 태그없음
680번째 태그없음
681번째 태그없음
683번째 태그없음
693번째 태그없음
704번째 태그없음
732번째 태그없음
736번째 태그없음
737번째 태그없음
759번째 태그없음
770번째 태그없음
779번째 태그없음
796번째 태그없음
799번째 태그없음
805번째 태그없음
807번째 태그없음
819번째 태그없음
821번째 태그없음
829번째 태그없음
861번째 태그없음
869번째 태그없음
877번째 태그없음
908번째 태그없음
909번째 태그없음
912번째 태그없음
942번째 태그없음
951번째 태그없음
956번째 태그없음
978번째 태그없음
985번째 태그없음
1025번째 태그없음
1028번째 태그없음
1029번째 태그없음
1037번째 태그없음
1053번째 태그없음
1058번째 태그없음
1074번째 태그없음
1079번째 태그없음
1087번째 태그없

In [40]:
val_copy2

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[OST, 기분전환, 디즈니, 애니메이션, 영화, 팝, 겨울, 추억, 휴식, 힐링]",118598,,"[349492, 422915, 547967, 42155, 422077, 648628...",1675,2019-05-27 14:14:33.000
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,"[발라드, 추억, 회상, 이별, 슬픔, 설렘, 사랑, 싸이월드, 밤, 새벽]",51464,,"[349492, 396828, 42155, 582252, 442014, 341513...",62,2008-06-21 23:26:22.000
3,"[발라드, 감성, 이별, 기분전환, 사랑, 카페, 드라이브, 잔잔한, 새벽, 휴식]",45144,,"[349492, 396828, 463173, 680366, 42155, 519391...",20,2017-10-30 18:15:43.000
4,"[CCM, 찬양, 은혜, 사랑, 예배, 국내ccm, 위로, 은혜로운, 찬송가, 교회]",79929,,"[349492, 582252, 341513, 396828, 509998, 42291...",20,2017-02-07 11:40:42.000
...,...,...,...,...,...,...
23010,"[발라드, 기분전환, 감성, 사랑, 이별, 새벽, 휴식, 잔잔한, 밤, 추억]",101722,,"[349492, 396828, 42155, 680366, 463173, 442014...",17,2015-12-17 14:06:05.000
23011,"[힐링, 휴식, 기분전환, 감성, 잔잔한, 발라드, 새벽, 사랑, 밤, 추억]",122127,,"[349492, 396828, 42155, 463173, 680366, 520093...",10,2020-04-16 21:35:44.000
23012,"[기분전환, 팝, 드라이브, 휴식, 힐링, 감성, Pop, 잔잔한, 팝송, 새벽]",77438,,"[349492, 463173, 396828, 42155, 520093, 680366...",0,2019-03-27 15:27:40.000
23013,"[클래식, 휴식, 힐링, 잔잔한, 팝, 추억, 기분전환, 올드팝, 감성, 피아노]",36231,,"[349492, 463173, 396828, 680366, 42155, 520093...",31,2015-11-18 11:49:09.000


In [41]:
# 저장
val_copy2.to_json('datas/val_copy2.json')

#### 다시 song에서 tag채우기 (2)

In [42]:
# 불러오기
val_copy2 = pd.read_json('datas/val_copy2.json')

In [43]:
# tag가 10개미만인 애들 다시 채우기
temp = val_copy2[val_copy2['tags'].apply(lambda x: len(x) < 10)]
temp.reset_index(drop=True, inplace=True)

In [44]:
from collections import Counter
from tqdm import notebook
# val_copy3 = val_copy2.copy()
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['songs'][i]) != 0:
        tags = {}
        # song_idx = j
        for j in range(0,len(temp['songs'][i])):
            if len(song_tags[song_tags['songs'] == int(temp['songs'][i][j])]) != 0:
                tag = song_tags[song_tags['songs'] == int(temp['songs'][i][j])]['tags_num'].values[0]
                tags = Counter(tags) + Counter(tag)
                temp['tags'][i] = list(dict(sorted(tags.items(), key = lambda x:x[1],reverse=True)[:10]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
val_copy3_temp = pd.concat([val_copy2,temp])
val_copy3_drpo = val_copy3_temp.drop_duplicates(['id'], keep = 'last')
val_copy3 = val_copy3_drpo.reset_index(drop = True)

In [46]:
# 저장
val_copy3.to_json('datas/val_copy3.json')

#### 채워진것 확인

In [199]:
val_copy3 = pd.read_json('datas/val_copy3.json')

In [200]:
# 노래가 100개 안채워진것
temp = val_copy3[val_copy3['songs'].apply(lambda x : len(x) < 100)]
temp

,tags,id,plylst_title,songs,like_cnt,updt_date
18340,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
18342,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
18343,"[발라드, 이별, 감성, 새벽, 잔잔한, 추억, 밤, 슬픔, 기분전환, 사랑]",2380,다시 생각나는 그 사람,"[556785, 651499, 1236, 600583, 253755, 537713,...",6,2016-03-19 23:53:20.000
18345,[],65114,"■■■■ 사랑,그리고이별 ■■■■",[],6,2010-10-27 10:34:34.000
18353,[],87700,마쉬멜로우같은 멜로우한 음악,[],6,2016-01-14 10:19:30.000
...,...,...,...,...,...,...
22996,[힘찬노래],61000,"세계를 정복한 가수, 방탄소년단",[],35,2018-06-09 18:49:29.000
23000,[],124704,가사의 의미와 뜻은모른다!! 오직 멜로디로만 선곡한 팝송!!,[],27,2016-02-05 12:31:59.000
23009,[],13045,＊카페 느낌 샹송♭,[],38,2011-07-12 00:58:39.000
23010,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000


In [201]:
# 정말 노래가 train에 없는가??
train_nosong = np.concatenate(temp['songs'].values)
train_nosong

array(['556785', '651499', '1236', ..., '452078', '439861', '251636'],
      dtype='<U32')

In [202]:
# 진짜 없네..
empty = []
for i in train_nosong:
    empty.append(song_tags[song_tags['songs'] == i])
pd.concat(empty)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,songs,tags,tags_num


In [203]:
len(empty)

2364

In [204]:
# 정말 태그가 train에 없는가?
train_notag = np.concatenate(temp['tags'].values)
train_notag

array(['발라드', '이별', '감성', '새벽', '잔잔한', '추억', '밤', '슬픔', '기분전환', '사랑',
       '멜군님께', '가비앤제이', '공짜', '광고ost', 'ThefirstLady', '퍼스트레이디',
       '나도내가의심될때', '몽환알엔비', '브리티쉬포크', '싸이키', '팝', '추억', '올드팝', '기분전환',
       '팝송', '힐링', '휴식', '재즈', '독서', '락', '재즈띵곡', 'remedy', '참아야겠지',
       '기분전환', '아이돌', '걸그룹', '댄스', '신나는', '드라이브', '봄', '사랑', '설렘', '운동',
       '포근한오후', '감성', '카페', '잔잔한', '휴식', '드라이브', '기분전환', '인디', '발라드', '밤',
       '사랑', '골때리는', '유이', '겁쟁이', '새벽', '밤', '잔잔한', '시규어로스', '몽환',
       'sigurros', '힐링', '락', '휴식', '우주', '감성', '인디', '사랑', '기분전환', '새벽',
       '카페', '잔잔한', '밤', '휴식', '발라드', '여자CCM', '여성CCM', '비비발디', '발라드',
       '감성', '기분전환', '사랑', '이별', '잔잔한', '휴식', '새벽', '추억', '힐링', '청춘2부작',
       '기분전환', '드라이브', 'Pop', '팝', '신나는', '팝송', '감성', '여행', '매장음악', 'EDM',
       '팝', '기분전환', 'Pop', '드라이브', '팝송', '감성', '신나는', '잔잔한', '휴식', '추억',
       '너만모르는노래', '가슴답답_할_때', '스트레스해소곡', '가슴뿡뚫리는곡', '아재라도', '신나는신나는',
       '빅데이터', '시원해지는노래', '틀딱', '추천한곡', '매장클래식', '외로울때', '지치고힘들때',
       

In [205]:
# 진짜 없네..
empty = []
for i in train_notag:
    empty.append(song_tags[song_tags['songs'] == i])
pd.concat(empty)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,songs,tags,tags_num


#### 플레이리스트의 명사로 태그를 채운 후 tag -> song을 채우자(3)

In [206]:
temp.reset_index(drop = True, inplace = True)

In [207]:
# 명사를 태그로?
from konlpy.tag import Okt
okt = Okt()
for i in range(0,len(temp)):
    temp['tags'][i] = okt.nouns(temp['plylst_title'][i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [208]:
temp

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
1,"[기분, 재즈, 달달, 하루]",142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
2,"[다시, 그, 사람]",2380,다시 생각나는 그 사람,"[556785, 651499, 1236, 600583, 253755, 537713,...",6,2016-03-19 23:53:20.000
3,"[사랑, 이별]",65114,"■■■■ 사랑,그리고이별 ■■■■",[],6,2010-10-27 10:34:34.000
4,"[마쉬멜로우, 멜로, 우한, 음악]",87700,마쉬멜로우같은 멜로우한 음악,[],6,2016-01-14 10:19:30.000
...,...,...,...,...,...,...
1915,"[세계, 정복, 가수, 방탄소년단]",61000,"세계를 정복한 가수, 방탄소년단",[],35,2018-06-09 18:49:29.000
1916,"[가사, 의미, 뜻, 오직, 멜로디, 로만, 선곡, 팝송]",124704,가사의 의미와 뜻은모른다!! 오직 멜로디로만 선곡한 팝송!!,[],27,2016-02-05 12:31:59.000
1917,"[카페, 느낌, 샹송]",13045,＊카페 느낌 샹송♭,[],38,2011-07-12 00:58:39.000
1918,"[컨트리, 황제, 조니, 캐시, 선, 레코드, 시절, 발표, 초기, 대표, 작]",32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000


In [209]:
# 다시한번 tag에서 송으로
# tag에서 song 채우기
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['tags'][i]) != 0:
        songs = {}
        # song_idx = j
        for j in range(0,len(temp['tags'][i])):
            if len(tag_songs[tag_songs['tags'] == temp['tags'][i][j]]) != 0:
                song = tag_songs[tag_songs['tags'] == temp['tags'][i][j]]['songs_num'].values[0]
                song = dict(itertools.islice(song.items(), 10000)) # 한 태그당 가져올 노래 갯수 지정
                songs = Counter(songs) + Counter(song)
                temp['songs'][i] = list(dict(sorted(songs.items(), key = lambda x:x[1],reverse=True)[:100]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [210]:
val_copy4_temp= pd.concat([val_copy3, temp])
val_copy4 = val_copy4_temp.drop_duplicates('id',keep = 'last')
val_copy4.reset_index(drop = True, inplace = True)

In [211]:
val_copy4.to_json('datas/val_copy4.json')

#### song에서tag 채우기(3-2)

In [212]:
val_copy4 = pd.read_json('datas/val_copy4.json')

In [213]:
# 일단 태그가 10개 미만인것 확인
temp = val_copy4[val_copy4['tags'].apply(lambda x : len(x)< 10)]
temp.reset_index(drop = True, inplace = True)

In [214]:
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['songs'][i]) != 0:
        tags = {}
        # song_idx = j
        for j in range(0,len(temp['songs'][i])):
            if len(song_tags[song_tags['songs'] == int(temp['songs'][i][j])]) != 0:
                tag = song_tags[song_tags['songs'] == int(temp['songs'][i][j])]['tags_num'].values[0]
                tags = Counter(tags) + Counter(tag)
                temp['tags'][i] = list(dict(sorted(tags.items(), key = lambda x:x[1],reverse=True)[:10]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [215]:
# 다 채웠음에도 불구하고 태그가 10개 미만인 애들
temp[temp['tags'].apply(lambda x : len(x) < 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],65571,,"[637545, 262476, 41104, 110067, 444876, 195364...",36,2019-03-22 14:49:42.000
1,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
6,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
19,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
47,[],6804,IMJMWEDAPLUGG,[],82,2020-04-15 18:40:49.000
...,...,...,...,...,...,...
1856,[],144555,,"[362641, 614449, 177918, 310240, 495650]",1,2015-11-09 14:13:20.000
1864,[한번],122585,한번들으면addiction되는OST,[],1,2014-08-20 21:52:34.000
1868,[],147048,HIT4864,[],0,2014-03-28 08:43:54.000
1877,[],99751,길게는6ㅏ9ㅙㅞㅏㅓㅏㅠㅏㅓㅠㅣㅓㅠㅣㅓㅠㅣㅏㅚㅑㅚㅑㅚㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟ,[],0,2018-11-27 09:12:45.000


In [216]:
val_5 = pd.concat([val_copy4,temp])
val_copy5 = val_5.drop_duplicates('id',keep = 'last')
val_copy5.reset_index(drop = True, inplace = True)

#### 다시확인

In [217]:
val_copy5[val_copy5['tags'].apply(lambda x :len(x) < 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
21101,[],65571,,"[637545, 262476, 41104, 110067, 444876, 195364...",36,2019-03-22 14:49:42.000
21102,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
21107,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
21120,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
21148,[],6804,IMJMWEDAPLUGG,[],82,2020-04-15 18:40:49.000
...,...,...,...,...,...,...
22957,[],144555,,"[362641, 614449, 177918, 310240, 495650]",1,2015-11-09 14:13:20.000
22965,[한번],122585,한번들으면addiction되는OST,[],1,2014-08-20 21:52:34.000
22969,[],147048,HIT4864,[],0,2014-03-28 08:43:54.000
22978,[],99751,길게는6ㅏ9ㅙㅞㅏㅓㅏㅠㅏㅓㅠㅣㅓㅠㅣㅓㅠㅣㅏㅚㅑㅚㅑㅚㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟ,[],0,2018-11-27 09:12:45.000


#### 저장하기

In [218]:
val_copy5.to_json('datas/val_copy5.json')

#### 불러오기

In [220]:
val_copy5 = pd.read_json('datas/val_copy5.json')

#### 바보같지만 다시 tag에서 song을 채워보자(4)

In [221]:
val_copy5[val_copy5['songs'].apply(lambda x :len(x) < 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
21102,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
21107,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
21120,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
21131,"[기분전환, 사랑, 설렘, 휴식, 드라이브, 발라드, 힐링, 감성, 새벽, 힙합]",11708,5회 아시아송페스티벌...★,"[13094, 135089, 188938, 69225, 627217, 474264,...",27,2008-10-05 12:14:29.000
21144,"[발라드, 감성, 사랑, 이별, 추억, 잔잔한, 기분전환, 새벽, 휴식, 회상]",31345,UK 프로젝트 vol.1,"[119979, 241793, 347553, 201207, 251472, 68912...",11,2008-06-26 14:49:38.000
...,...,...,...,...,...,...
22978,[],99751,길게는6ㅏ9ㅙㅞㅏㅓㅏㅠㅏㅓㅠㅣㅓㅠㅣㅓㅠㅣㅏㅚㅑㅚㅑㅚㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟ,[],0,2018-11-27 09:12:45.000
22980,"[OST, 새벽, 인터스텔라, 영화, 밤, 잔잔한, 휴식, 힐링, 영화음악, 기분전환]",48175,인터스텔라OST 한스짐머,"[234710, 303681, 58196, 294896, 480200, 54348,...",88,2014-12-29 12:14:38.000
22994,"[디즈니, 체드, 디센던츠, 제이, 제인, 말, 카를로스, 이비, 오드리, 여행]",116719,보고싶다는 말조차,"[57875, 374409, 614567, 403758, 360272, 391008...",2,2017-07-01 22:51:08.000
23002,[],22014,,[],3,2015-05-25 17:10:52.000


In [222]:
temp = val_copy5[val_copy5['songs'].apply(lambda x :len(x) < 100)]
temp.reset_index(drop = True, inplace = True)

In [223]:
# 다시한번 tag에서 송으로
# tag에서 song 채우기
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['tags'][i]) != 0:
        songs = {}
        # song_idx = j
        for j in range(0,len(temp['tags'][i])):
            if len(tag_songs[tag_songs['tags'] == temp['tags'][i][j]]) != 0:
                song = tag_songs[tag_songs['tags'] == temp['tags'][i][j]]['songs_num'].values[0]
                song = dict(itertools.islice(song.items(), 10000)) # 한 태그당 가져올 노래 갯수 지정
                songs = Counter(songs) + Counter(song)
                temp['songs'][i] = list(dict(sorted(songs.items(), key = lambda x:x[1],reverse=True)[:100]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [224]:
temp[temp['songs'].apply(lambda x :len(x) < 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
1,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
2,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
5,[],6804,IMJMWEDAPLUGG,[],82,2020-04-15 18:40:49.000
6,[],97427,My POP 2,[],6,2016-06-26 20:43:32.000
...,...,...,...,...,...,...
359,[],144555,,"[362641, 614449, 177918, 310240, 495650]",1,2015-11-09 14:13:20.000
360,[한번],122585,한번들으면addiction되는OST,[],1,2014-08-20 21:52:34.000
361,[],147048,HIT4864,[],0,2014-03-28 08:43:54.000
363,[],99751,길게는6ㅏ9ㅙㅞㅏㅓㅏㅠㅏㅓㅠㅣㅓㅠㅣㅓㅠㅣㅏㅚㅑㅚㅑㅚㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟ,[],0,2018-11-27 09:12:45.000


In [225]:
val_6 = pd.concat([val_copy5,temp])
val_copy6 = val_6.drop_duplicates('id',keep = 'last')
val_copy6.reset_index(drop = True, inplace = True)

#### 저장하기

In [226]:
val_copy6.to_json('datas/val_copy6.json')

#### 다시 song에서 tag를 채워보자(4-2)

In [227]:
val_copy6 = pd.read_json('datas/val_copy6.json')
val_copy6

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[OST, 기분전환, 디즈니, 애니메이션, 영화, 팝, 겨울, 추억, 휴식, 힐링]",118598,,"[349492, 422915, 547967, 42155, 422077, 648628...",1675,2019-05-27 14:14:33.000
1,"[발라드, 추억, 회상, 이별, 슬픔, 설렘, 사랑, 싸이월드, 밤, 새벽]",51464,,"[349492, 396828, 42155, 582252, 442014, 341513...",62,2008-06-21 23:26:22.000
2,"[발라드, 감성, 이별, 기분전환, 사랑, 카페, 드라이브, 잔잔한, 새벽, 휴식]",45144,,"[349492, 396828, 463173, 680366, 42155, 519391...",20,2017-10-30 18:15:43.000
3,"[CCM, 찬양, 은혜, 사랑, 예배, 국내ccm, 위로, 은혜로운, 찬송가, 교회]",79929,,"[349492, 582252, 341513, 396828, 509998, 42291...",20,2017-02-07 11:40:42.000
4,"[Pop, 기분전환, 드라이브, 빌보드, 빌보드Hot50, 트렌디, 스트레스, 빌보...",138538,,"[493762, 581799, 519391, 640657, 15124, 146989...",2,2019-06-12 17:43:35.000
...,...,...,...,...,...,...
23010,[],99751,길게는6ㅏ9ㅙㅞㅏㅓㅏㅠㅏㅓㅠㅣㅓㅠㅣㅓㅠㅣㅏㅚㅑㅚㅑㅚㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟ,[],0,2018-11-27 09:12:45.000
23011,"[OST, 새벽, 인터스텔라, 영화, 밤, 잔잔한, 휴식, 힐링, 영화음악, 기분전환]",48175,인터스텔라OST 한스짐머,"[349492, 396828, 42155, 463173, 520093, 422915...",88,2014-12-29 12:14:38.000
23012,"[디즈니, 체드, 디센던츠, 제이, 제인, 말, 카를로스, 이비, 오드리, 여행]",116719,보고싶다는 말조차,"[205179, 346619, 581799, 699874, 146989, 19191...",2,2017-07-01 22:51:08.000
23013,[],22014,,[],3,2015-05-25 17:10:52.000


In [228]:
val_copy6[val_copy6['tags'].apply(lambda x :len(x) < 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
21101,[],65571,,"[637545, 262476, 41104, 110067, 444876, 195364...",36,2019-03-22 14:49:42.000
22647,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
22648,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
22649,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
22652,[],6804,IMJMWEDAPLUGG,[],82,2020-04-15 18:40:49.000
...,...,...,...,...,...,...
23006,[],144555,,"[362641, 614449, 177918, 310240, 495650]",1,2015-11-09 14:13:20.000
23007,[한번],122585,한번들으면addiction되는OST,[],1,2014-08-20 21:52:34.000
23008,[],147048,HIT4864,[],0,2014-03-28 08:43:54.000
23010,[],99751,길게는6ㅏ9ㅙㅞㅏㅓㅏㅠㅏㅓㅠㅣㅓㅠㅣㅓㅠㅣㅏㅚㅑㅚㅑㅚㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟ,[],0,2018-11-27 09:12:45.000


In [229]:
temp = val_copy6[val_copy6['tags'].apply(lambda x :len(x) < 10)]
temp.reset_index(drop = True, inplace = True)

In [230]:
# 하기전이랑 하고난 뒤랑 똑같다 의미가 없음 이제
temp[temp['tags'].apply(lambda x :len(x) < 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],65571,,"[637545, 262476, 41104, 110067, 444876, 195364...",36,2019-03-22 14:49:42.000
1,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
2,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
3,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
4,[],6804,IMJMWEDAPLUGG,[],82,2020-04-15 18:40:49.000
...,...,...,...,...,...,...
319,[],144555,,"[362641, 614449, 177918, 310240, 495650]",1,2015-11-09 14:13:20.000
320,[한번],122585,한번들으면addiction되는OST,[],1,2014-08-20 21:52:34.000
321,[],147048,HIT4864,[],0,2014-03-28 08:43:54.000
322,[],99751,길게는6ㅏ9ㅙㅞㅏㅓㅏㅠㅏㅓㅠㅣㅓㅠㅣㅓㅠㅣㅏㅚㅑㅚㅑㅚㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟㅏㅟ,[],0,2018-11-27 09:12:45.000


In [231]:
# title이 영어인것 확인해보기
temp2 = temp[temp['plylst_title'].apply(lambda x : len(x) !=0)]
temp2.reset_index(drop = True, inplace = True)
temp2

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[앨리스, 테이블]",131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
1,[],6804,IMJMWEDAPLUGG,[],82,2020-04-15 18:40:49.000
2,[],97427,My POP 2,[],6,2016-06-26 20:43:32.000
3,[],116623,Painting Of Body Party,[],47,2014-05-21 14:53:44.000
4,[],138702,지겹게 들었다..2,[],20,2007-02-18 14:56:48.000
...,...,...,...,...,...,...
221,[],104205,SMTOWN,[],2,2012-07-07 15:59:31.000
222,[],90670,deep & dope,[],22,2016-04-16 18:25:52.000
223,[한번],122585,한번들으면addiction되는OST,[],1,2014-08-20 21:52:34.000
224,[],147048,HIT4864,[],0,2014-03-28 08:43:54.000


In [232]:
# 영어만 남기고 삭제
import re
for i in range(0,len(temp2)):
    only_english = re.sub('[^a-zA-Z]', ' ', temp2['plylst_title'][i])
    temp2['tags'][i]= only_english.lower()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [233]:
# 확인
temp = temp2[temp2['tags'].apply(lambda x :len(x) !=0)]
temp.reset_index(drop = True, inplace = True)
temp

,tags,id,plylst_title,songs,like_cnt,updt_date
0,,131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
1,imjmwedaplugg,6804,IMJMWEDAPLUGG,[],82,2020-04-15 18:40:49.000
2,my pop,97427,My POP 2,[],6,2016-06-26 20:43:32.000
3,painting of body party,116623,Painting Of Body Party,[],47,2014-05-21 14:53:44.000
4,,138702,지겹게 들었다..2,[],20,2007-02-18 14:56:48.000
...,...,...,...,...,...,...
221,smtown,104205,SMTOWN,[],2,2012-07-07 15:59:31.000
222,deep dope,90670,deep & dope,[],22,2016-04-16 18:25:52.000
223,addiction ost,122585,한번들으면addiction되는OST,[],1,2014-08-20 21:52:34.000
224,hit,147048,HIT4864,[],0,2014-03-28 08:43:54.000


#### 일단 이 playtitle로 노래를 채워본다. (5)

In [234]:
import itertools
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['tags'][i]) != 0:
        songs = {}
        # song_idx = j
        for j in range(0,len(temp['tags'][i])):
            if len(tag_songs[tag_songs['tags'] == temp['tags'][i][j]]) != 0:
                song = tag_songs[tag_songs['tags'] == temp['tags'][i][j]]['songs_num'].values[0]
                song = dict(itertools.islice(song.items(), 10000)) # 한 태그당 가져올 노래 갯수 지정
                songs = Counter(songs) + Counter(song)
                temp['songs'][i] = list(dict(sorted(songs.items(), key = lambda x:x[1],reverse=True)[:100]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### 다시 song에서 tag를 만든다 (5-2)

In [235]:
import itertools
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['songs'][i]) != 0:
        tags = {}
        # song_idx = j
        for j in range(0,len(temp['songs'][i])):
            if len(song_tags[song_tags['songs'] == int(temp['songs'][i][j])]) != 0:
                tag = song_tags[song_tags['songs'] == int(temp['songs'][i][j])]['tags_num'].values[0]
                tags = Counter(tags) + Counter(tag)
                temp['tags'][i] = list(dict(sorted(tags.items(), key = lambda x:x[1],reverse=True)[:10]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [236]:
# tag 10개 확인
temp[temp['tags'].apply(lambda x:len(x) == 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
1,"[힙합, 기분전환, 드라이브, 랩, 감성, 새벽, 팝, 밤, 신나는, 알앤비]",6804,IMJMWEDAPLUGG,"[419602, 424738, 512218, 625463, 518916, 42524...",82,2020-04-15 18:40:49.000
2,"[힙합, 기분전환, 드라이브, 랩, 감성, 새벽, 팝, 밤, 신나는, 알앤비]",97427,My POP 2,"[424738, 512218, 625463, 518916, 425245, 39612...",6,2016-06-26 20:43:32.000
3,"[발라드, 추억, 회상, 기분전환, 이별, 감성, 사랑, 휴식, 겨울, 슬픔]",116623,Painting Of Body Party,"[619695, 21901, 477095, 48677, 40051, 413698, ...",47,2014-05-21 14:53:44.000
4,,138702,지겹게 들었다..2,[],20,2007-02-18 14:56:48.000
5,"[기분전환, 감성, 사랑, 드라이브, 휴식, 힙합, 새벽, 발라드, 카페, 힐링]",64027,pop list,"[301494, 697364, 255485, 623767, 160160, 46284...",42,2015-12-10 12:21:31.000
...,...,...,...,...,...,...
219,"[감성, 기분전환, 드라이브, 카페, 힙합, 새벽, 휴식, 밤, 사랑, 잔잔한]",2314,공개] DUET,"[672319, 116250, 141248, 46284, 537571, 111933...",5,2011-05-06 13:40:09.000
220,i like it,131826,I Like it!,[],30,2014-03-30 10:06:57.000
221,"[기분전환, 감성, 힙합, 사랑, 드라이브, 휴식, 새벽, 발라드, 힐링, 밤]",104205,SMTOWN,"[301494, 697364, 255485, 623767, 160160, 46284...",2,2012-07-07 15:59:31.000
222,"[감성, 기분전환, 드라이브, 카페, 힙합, 새벽, 휴식, 밤, 사랑, 잔잔한]",90670,deep & dope,"[672319, 116250, 141248, 46284, 537571, 111933...",22,2016-04-16 18:25:52.000


In [237]:
# tag가 10개가 아닌거, 11개도 보인다.
temp[temp['tags'].apply(lambda x : len(x) > 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
21,going to jazz bar,98543,Going to .. Jazz Bar,[],5,2005-09-20 10:36:02.000
23,gentle pop at late night,2652,Gentle pop at late night,[],469,2015-07-12 10:22:35.000
50,,147655,그저그런나날이라도 한결같이평온하게,[],14,2011-02-14 11:38:26.000
55,pop,18781,★★ 상쾌한 POP ★★,[],0,2006-01-12 22:53:24.000
68,,52251,#휴일 증후군 날려보내기..!,[],14,2015-12-28 18:15:01.000
76,,133485,니가있던그자리에....,[],1,2006-12-07 00:32:55.000
84,,84125,설레이고 싶을때 들어봐요❤,[],10,2016-03-22 22:46:56.000
96,fun of jazz,112437,Fun Of Jazz,[],21,2008-04-23 21:13:20.000
104,,52178,편히 쉬다 가는 곳.,[],2,2013-03-21 22:38:38.000
109,club lounge,77696,"Club, Lounge",[],18,2014-11-24 12:49:57.000


#### 다시 tag에서song을 넣기

In [238]:
import itertools
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['tags'][i]) != 0:
        songs = {}
        # song_idx = j
        for j in range(0,len(temp['tags'][i])):
            if len(tag_songs[tag_songs['tags'] == temp['tags'][i][j]]) != 0:
                song = tag_songs[tag_songs['tags'] == temp['tags'][i][j]]['songs_num'].values[0]
                song = dict(itertools.islice(song.items(), 10000)) # 한 태그당 가져올 노래 갯수 지정
                songs = Counter(songs) + Counter(song)
                temp['songs'][i] = list(dict(sorted(songs.items(), key = lambda x:x[1],reverse=True)[:100]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [239]:
val_7 = pd.concat([val_copy6,temp])
val_copy7 = val_7.drop_duplicates('id',keep = 'last')
val_copy7.reset_index(drop = True, inplace = True)

#### 저장하기

In [240]:
val_copy7.to_json('datas/val_copy7.json')

#### 불러오기

In [241]:
val_copy7 =pd.read_json('datas/val_copy7.json')

In [242]:
# tag가 10개가 아닌거, 11개도 보인다.
val_copy7[val_copy7['tags'].apply(lambda x : len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
21097,"[더, 뮤지션, 워, 원, 황민, 현, 첫, 은, 두번째, 곡, 시펑]",1963,더뮤지션 워너원 황민현의 첫번째에 이은 두번째 PICK 곡 듣고 시펑?,"[459989, 413422, 642526, 635501, 468885, 31081...",5,2017-10-10 14:25:22.000
21099,"[주의, 코로나, 집, 순이, 만렙, 재생, 시, 방, 구석, 국, 클럽]",15439,*주의* 코로나로 집순이만렙찍어도 재생 시 방구석이 와국 클럽으로 변함,"[310151, 425668, 31529, 547427, 413952, 649359...",1,2020-04-22 15:37:51.000
21100,"[컨트리, 황제, 조니, 캐시, 선, 레코드, 시절, 발표, 초기, 대표, 작]",32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,"[675593, 517472, 669151, 432499, 652045, 38288...",28,2019-06-17 14:22:48.000
21101,[],65571,,"[637545, 262476, 41104, 110067, 444876, 195364...",36,2019-03-22 14:49:42.000
22647,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
...,...,...,...,...,...,...
23005,,90407,"푸른새벽, 그리고",[],11,2017-09-05 23:11:28.000
23006,,95396,ㅈㅈㅈㅈ,[],0,2016-10-12 11:49:32.000
23007,,6666,1000,[],23,2017-11-23 11:53:24.000
23013,hit,147048,HIT4864,[],0,2014-03-28 08:43:54.000


In [243]:
# song이 100개가 아닌거
val_copy7[val_copy7['songs'].apply(lambda x : len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
22647,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
22648,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
22651,[],52149,,"[638496, 620895, 643877, 170671, 78488, 307555...",1,2020-02-03 16:45:42.000
22652,[],114714,,"[350110, 191714, 515717, 593742, 407145, 24027...",17,2019-12-19 14:47:44.000
22653,[],104922,,"[316484, 53931, 539725, 204829]",14,2016-07-22 00:42:23.000
...,...,...,...,...,...,...
23006,,95396,ㅈㅈㅈㅈ,[],0,2016-10-12 11:49:32.000
23007,,6666,1000,[],23,2017-11-23 11:53:24.000
23009,i like it,131826,I Like it!,[],30,2014-03-30 10:06:57.000
23013,hit,147048,HIT4864,[],0,2014-03-28 08:43:54.000


In [244]:
temp = val_copy7[val_copy7['tags'].apply(lambda x : len(x) != 10)]
temp.reset_index(drop = True, inplace = True)

#### 일단 tag에서 song을 채우자(6)

In [245]:
import itertools
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['songs'][i]) != 0:
        tags = {}
        # song_idx = j
        for j in range(0,len(temp['songs'][i])):
            if len(song_tags[song_tags['songs'] == int(temp['songs'][i][j])]) != 0:
                tag = song_tags[song_tags['songs'] == int(temp['songs'][i][j])]['tags_num'].values[0]
                tags = Counter(tags) + Counter(tag)
                temp['tags'][i] = list(dict(sorted(tags.items(), key = lambda x:x[1],reverse=True)[:10]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [247]:
# 7개 채워짐
temp[temp['tags'].apply(lambda x : len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
3,[],65571,,"[637545, 262476, 41104, 110067, 444876, 195364...",36,2019-03-22 14:49:42.000
4,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
5,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
6,[],52149,,"[638496, 620895, 643877, 170671, 78488, 307555...",1,2020-02-03 16:45:42.000
7,[],114714,,"[350110, 191714, 515717, 593742, 407145, 24027...",17,2019-12-19 14:47:44.000
...,...,...,...,...,...,...
199,,90407,"푸른새벽, 그리고",[],11,2017-09-05 23:11:28.000
200,,95396,ㅈㅈㅈㅈ,[],0,2016-10-12 11:49:32.000
201,,6666,1000,[],23,2017-11-23 11:53:24.000
202,hit,147048,HIT4864,[],0,2014-03-28 08:43:54.000


In [248]:
# song이 100개가 아닌거
temp[temp['songs'].apply(lambda x : len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
4,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
5,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
6,[],52149,,"[638496, 620895, 643877, 170671, 78488, 307555...",1,2020-02-03 16:45:42.000
7,[],114714,,"[350110, 191714, 515717, 593742, 407145, 24027...",17,2019-12-19 14:47:44.000
8,[],104922,,"[316484, 53931, 539725, 204829]",14,2016-07-22 00:42:23.000
...,...,...,...,...,...,...
199,,90407,"푸른새벽, 그리고",[],11,2017-09-05 23:11:28.000
200,,95396,ㅈㅈㅈㅈ,[],0,2016-10-12 11:49:32.000
201,,6666,1000,[],23,2017-11-23 11:53:24.000
202,hit,147048,HIT4864,[],0,2014-03-28 08:43:54.000


#### 다시 song에서 tag를 채우자 (6-2)

In [249]:
import itertools
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['songs'][i]) != 0:
        tags = {}
        # song_idx = j
        for j in range(0,len(temp['songs'][i])):
            if len(song_tags[song_tags['songs'] == int(temp['songs'][i][j])]) != 0:
                tag = song_tags[song_tags['songs'] == int(temp['songs'][i][j])]['tags_num'].values[0]
                tags = Counter(tags) + Counter(tag)
                temp['tags'][i] = list(dict(sorted(tags.items(), key = lambda x:x[1],reverse=True)[:10]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [250]:
# 똑같음 이제는 끝인듯..
temp[temp['songs'].apply(lambda x : len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
4,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
5,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
6,[],52149,,"[638496, 620895, 643877, 170671, 78488, 307555...",1,2020-02-03 16:45:42.000
7,[],114714,,"[350110, 191714, 515717, 593742, 407145, 24027...",17,2019-12-19 14:47:44.000
8,[],104922,,"[316484, 53931, 539725, 204829]",14,2016-07-22 00:42:23.000
...,...,...,...,...,...,...
199,,90407,"푸른새벽, 그리고",[],11,2017-09-05 23:11:28.000
200,,95396,ㅈㅈㅈㅈ,[],0,2016-10-12 11:49:32.000
201,,6666,1000,[],23,2017-11-23 11:53:24.000
202,hit,147048,HIT4864,[],0,2014-03-28 08:43:54.000


In [251]:
val_8 = pd.concat([val_copy7,temp])
val_copy8 = val_8.drop_duplicates('id',keep = 'last')
val_copy8.reset_index(drop = True, inplace = True)

#### 저장하기

In [252]:
val_copy8.to_json('datas/val_copy8.json')

#### 불러오기

In [253]:
val_copy8 = pd.read_json('datas/val_copy8.json')

In [254]:
# song이 100개 아닌거, 앞이랑 똑같음 이제는 의미가 없는듯..
val_copy8[val_copy8['songs'].apply(lambda x : len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
22691,,138702,지겹게 들었다..2,[],20,2007-02-18 14:56:48.000
22706,up up up,68797,up up up!!,[],3,2006-11-29 10:14:56.000
22755,,5216,20160920 1,[],0,2016-09-20 18:39:36.000
22760,,45977,좋아요 너무 좋아요,[],3,2017-11-13 16:01:57.000
22763,,124962,테크토닉에 빠져들다,[],2397,2010-01-19 14:55:01.000
...,...,...,...,...,...,...
23010,,90407,"푸른새벽, 그리고",[],11,2017-09-05 23:11:28.000
23011,,95396,ㅈㅈㅈㅈ,[],0,2016-10-12 11:49:32.000
23012,,6666,1000,[],23,2017-11-23 11:53:24.000
23013,hit,147048,HIT4864,[],0,2014-03-28 08:43:54.000


In [257]:
# tag가 10개 이상인거, 영어단어인데 일단, 한국어로 바꿔주면 태그가 나올거같은데?
val_copy8[val_copy8['tags'].apply(lambda x : len(x) > 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
22920,going to jazz bar,98543,Going to .. Jazz Bar,[],5,2005-09-20 10:36:02.000
22922,gentle pop at late night,2652,Gentle pop at late night,[],469,2015-07-12 10:22:35.000
22936,,147655,그저그런나날이라도 한결같이평온하게,[],14,2011-02-14 11:38:26.000
22938,pop,18781,★★ 상쾌한 POP ★★,[],0,2006-01-12 22:53:24.000
22944,,52251,#휴일 증후군 날려보내기..!,[],14,2015-12-28 18:15:01.000
22946,,133485,니가있던그자리에....,[],1,2006-12-07 00:32:55.000
22950,,84125,설레이고 싶을때 들어봐요❤,[],10,2016-03-22 22:46:56.000
22956,fun of jazz,112437,Fun Of Jazz,[],21,2008-04-23 21:13:20.000
22960,,52178,편히 쉬다 가는 곳.,[],2,2013-03-21 22:38:38.000
22964,club lounge,77696,"Club, Lounge",[],18,2014-11-24 12:49:57.000


In [258]:
temp = val_copy8[val_copy8['tags'].apply(lambda x : len(x) > 10 )]
temp.reset_index(drop = True, inplace =True)

In [259]:
temp

,tags,id,plylst_title,songs,like_cnt,updt_date
0,going to jazz bar,98543,Going to .. Jazz Bar,[],5,2005-09-20 10:36:02.000
1,gentle pop at late night,2652,Gentle pop at late night,[],469,2015-07-12 10:22:35.000
2,,147655,그저그런나날이라도 한결같이평온하게,[],14,2011-02-14 11:38:26.000
3,pop,18781,★★ 상쾌한 POP ★★,[],0,2006-01-12 22:53:24.000
4,,52251,#휴일 증후군 날려보내기..!,[],14,2015-12-28 18:15:01.000
5,,133485,니가있던그자리에....,[],1,2006-12-07 00:32:55.000
6,,84125,설레이고 싶을때 들어봐요❤,[],10,2016-03-22 22:46:56.000
7,fun of jazz,112437,Fun Of Jazz,[],21,2008-04-23 21:13:20.000
8,,52178,편히 쉬다 가는 곳.,[],2,2013-03-21 22:38:38.000
9,club lounge,77696,"Club, Lounge",[],18,2014-11-24 12:49:57.000


In [260]:
temp['tags'][0] = ['재즈']
temp['tags'][1] = ['늦은밤', 'Pop', '팝']
temp['tags'][3] = ['Pop', '팝']
temp['tags'][7] = ['재즈']
temp['tags'][9] = ['Club', '클럽']
temp['tags'][12] = ['락', '록', '락앤롤']
temp['tags'][13] = ['제이팝']
temp['tags'][17] = ['락', '록', '락앤롤']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [261]:
temp

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[재즈],98543,Going to .. Jazz Bar,[],5,2005-09-20 10:36:02.000
1,"[늦은밤, Pop, 팝]",2652,Gentle pop at late night,[],469,2015-07-12 10:22:35.000
2,,147655,그저그런나날이라도 한결같이평온하게,[],14,2011-02-14 11:38:26.000
3,"[Pop, 팝]",18781,★★ 상쾌한 POP ★★,[],0,2006-01-12 22:53:24.000
4,,52251,#휴일 증후군 날려보내기..!,[],14,2015-12-28 18:15:01.000
5,,133485,니가있던그자리에....,[],1,2006-12-07 00:32:55.000
6,,84125,설레이고 싶을때 들어봐요❤,[],10,2016-03-22 22:46:56.000
7,[재즈],112437,Fun Of Jazz,[],21,2008-04-23 21:13:20.000
8,,52178,편히 쉬다 가는 곳.,[],2,2013-03-21 22:38:38.000
9,"[Club, 클럽]",77696,"Club, Lounge",[],18,2014-11-24 12:49:57.000


#### 변경한 태그에서 곡을 채우자

In [262]:
import itertools
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['tags'][i]) != 0:
        songs = {}
        # song_idx = j
        for j in range(0,len(temp['tags'][i])):
            if len(tag_songs[tag_songs['tags'] == temp['tags'][i][j]]) != 0:
                song = tag_songs[tag_songs['tags'] == temp['tags'][i][j]]['songs_num'].values[0]
                song = dict(itertools.islice(song.items(), 10000)) # 한 태그당 가져올 노래 갯수 지정
                songs = Counter(songs) + Counter(song)
                temp['songs'][i] = list(dict(sorted(songs.items(), key = lambda x:x[1],reverse=True)[:100]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [263]:
temp

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[재즈],98543,Going to .. Jazz Bar,"[92908, 705515, 321724, 650367, 356141, 222610...",5,2005-09-20 10:36:02.000
1,"[늦은밤, Pop, 팝]",2652,Gentle pop at late night,"[493762, 581799, 15124, 140867, 625875, 464051...",469,2015-07-12 10:22:35.000
2,,147655,그저그런나날이라도 한결같이평온하게,[],14,2011-02-14 11:38:26.000
3,"[Pop, 팝]",18781,★★ 상쾌한 POP ★★,"[493762, 581799, 140867, 15124, 625875, 464051...",0,2006-01-12 22:53:24.000
4,,52251,#휴일 증후군 날려보내기..!,[],14,2015-12-28 18:15:01.000
5,,133485,니가있던그자리에....,[],1,2006-12-07 00:32:55.000
6,,84125,설레이고 싶을때 들어봐요❤,[],10,2016-03-22 22:46:56.000
7,[재즈],112437,Fun Of Jazz,"[92908, 705515, 321724, 650367, 356141, 222610...",21,2008-04-23 21:13:20.000
8,,52178,편히 쉬다 가는 곳.,[],2,2013-03-21 22:38:38.000
9,"[Club, 클럽]",77696,"Club, Lounge","[310151, 31529, 425668, 547427, 413952, 649359...",18,2014-11-24 12:49:57.000


#### 다시 곡에서 태그를 채우자

In [264]:
import itertools
from collections import Counter
from tqdm import notebook
# df_indx = i
for i in notebook.tqdm(range(0,len(temp))):
    if len(temp['songs'][i]) != 0:
        tags = {}
        # song_idx = j
        for j in range(0,len(temp['songs'][i])):
            if len(song_tags[song_tags['songs'] == int(temp['songs'][i][j])]) != 0:
                tag = song_tags[song_tags['songs'] == int(temp['songs'][i][j])]['tags_num'].values[0]
                tags = Counter(tags) + Counter(tag)
                temp['tags'][i] = list(dict(sorted(tags.items(), key = lambda x:x[1],reverse=True)[:10]).keys())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [267]:
# 태그는 잘채워짐
temp[temp['tags'].apply(lambda x : len(x)  == 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[재즈, 카페, 휴식, 힐링, 기분전환, 잔잔한, 감성, 새벽, 팝, Jazz]",98543,Going to .. Jazz Bar,"[92908, 705515, 321724, 650367, 356141, 222610...",5,2005-09-20 10:36:02.000
1,"[기분전환, 팝, 드라이브, Pop, 신나는, 팝송, 감성, 휴식, 여행, 힐링]",2652,Gentle pop at late night,"[493762, 581799, 15124, 140867, 625875, 464051...",469,2015-07-12 10:22:35.000
3,"[기분전환, 팝, 드라이브, Pop, 신나는, 팝송, 감성, 휴식, 여행, 힐링]",18781,★★ 상쾌한 POP ★★,"[493762, 581799, 140867, 15124, 625875, 464051...",0,2006-01-12 22:53:24.000
7,"[재즈, 카페, 휴식, 힐링, 기분전환, 잔잔한, 감성, 새벽, 팝, Jazz]",112437,Fun Of Jazz,"[92908, 705515, 321724, 650367, 356141, 222610...",21,2008-04-23 21:13:20.000
9,"[클럽, 기분전환, 드라이브, 일렉, EDM, 신나는, 운동, 스트레스, 팝, Pop]",77696,"Club, Lounge","[310151, 31529, 425668, 547427, 413952, 649359...",18,2014-11-24 12:49:57.000
12,"[락, 기분전환, 팝, 휴식, 새벽, 드라이브, 힐링, 잔잔한, 밤, 감성]",130953,"Rock, 어렵지 않아요~","[532771, 439301, 596414, 267159, 142557, 17523...",167,2016-10-26 08:42:58.000
13,"[JPOP, 기분전환, 제이팝, 일본, 일본노래, 감성, 일본음악, 휴식, 드라이브...",121224,들으면 아! 하고 감탄할 J-POP,"[168744, 653791, 126691, 632189, 624957, 82815...",6,2016-07-20 18:05:16.000
17,"[락, 기분전환, 팝, 휴식, 새벽, 드라이브, 힐링, 잔잔한, 밤, 감성]",31947,Rock'n Roll,"[532771, 439301, 596414, 267159, 142557, 17523...",2,2009-01-13 00:22:32.000


In [268]:
# song도 잘채워짐
temp[temp['songs'].apply(lambda x : len(x)  == 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[재즈, 카페, 휴식, 힐링, 기분전환, 잔잔한, 감성, 새벽, 팝, Jazz]",98543,Going to .. Jazz Bar,"[92908, 705515, 321724, 650367, 356141, 222610...",5,2005-09-20 10:36:02.000
1,"[기분전환, 팝, 드라이브, Pop, 신나는, 팝송, 감성, 휴식, 여행, 힐링]",2652,Gentle pop at late night,"[493762, 581799, 15124, 140867, 625875, 464051...",469,2015-07-12 10:22:35.000
3,"[기분전환, 팝, 드라이브, Pop, 신나는, 팝송, 감성, 휴식, 여행, 힐링]",18781,★★ 상쾌한 POP ★★,"[493762, 581799, 140867, 15124, 625875, 464051...",0,2006-01-12 22:53:24.000
7,"[재즈, 카페, 휴식, 힐링, 기분전환, 잔잔한, 감성, 새벽, 팝, Jazz]",112437,Fun Of Jazz,"[92908, 705515, 321724, 650367, 356141, 222610...",21,2008-04-23 21:13:20.000
9,"[클럽, 기분전환, 드라이브, 일렉, EDM, 신나는, 운동, 스트레스, 팝, Pop]",77696,"Club, Lounge","[310151, 31529, 425668, 547427, 413952, 649359...",18,2014-11-24 12:49:57.000
12,"[락, 기분전환, 팝, 휴식, 새벽, 드라이브, 힐링, 잔잔한, 밤, 감성]",130953,"Rock, 어렵지 않아요~","[532771, 439301, 596414, 267159, 142557, 17523...",167,2016-10-26 08:42:58.000
13,"[JPOP, 기분전환, 제이팝, 일본, 일본노래, 감성, 일본음악, 휴식, 드라이브...",121224,들으면 아! 하고 감탄할 J-POP,"[168744, 653791, 126691, 632189, 624957, 82815...",6,2016-07-20 18:05:16.000
17,"[락, 기분전환, 팝, 휴식, 새벽, 드라이브, 힐링, 잔잔한, 밤, 감성]",31947,Rock'n Roll,"[532771, 439301, 596414, 267159, 142557, 17523...",2,2009-01-13 00:22:32.000


In [269]:
val_9 = pd.concat([val_copy8,temp])
val_copy9 = val_9.drop_duplicates('id',keep = 'last')
val_copy9.reset_index(drop = True, inplace = True)

#### 저장하기

In [270]:
val_copy9.to_json('datas/val_copy9.json')

#### 불러오기

In [271]:
val_copy9 = pd.read_json('datas/val_copy9.json')

In [272]:
# 노래가 100개 아니면서 0개 이상인거, 즉 노래가 다 안채워진것
val_copy9[val_copy9['songs'].apply(lambda x : len(x) != 100 and len(x) > 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date
22815,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000
22816,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000
22817,[],52149,,"[638496, 620895, 643877, 170671, 78488, 307555...",1,2020-02-03 16:45:42.000
22818,[],114714,,"[350110, 191714, 515717, 593742, 407145, 24027...",17,2019-12-19 14:47:44.000
22819,[],104922,,"[316484, 53931, 539725, 204829]",14,2016-07-22 00:42:23.000
...,...,...,...,...,...,...
22906,[],146744,,"[96017, 198264]",0,2015-10-02 14:50:11.000
22907,[],63799,,"[389876, 255641, 432475, 90103, 239292, 18542,...",2,2014-09-28 13:46:15.000
22908,[],108712,,"[596632, 374760, 582554, 503780, 473771]",1,2020-04-03 17:03:31.000
22909,[],86094,,"[430360, 180264, 371559, 361053, 229181, 98991...",14,2018-03-12 14:38:38.000


In [273]:
# 얘내를 100개로 채우자
temp = val_copy9[val_copy9['songs'].apply(lambda x : len(x) != 100 and len(x) > 0)]
temp.reset_index(drop = True, inplace = True)

In [274]:
import random
random.seed(13)
song_list = list(song_tags['songs'])
for i in range(0, len(temp)):
    if len(temp['tags'][i]) == 10:
        s = list(map(str,random.sample(song_list, 100 - len(temp['songs'][i]))))
        temp['songs'][i] = list(set(s + temp['songs'][i]))
    else:
        temp['songs'][i] = []
        s = list(map(str,random.sample(song_list, 100)))
        temp['songs'][i] = list(set(s + temp['songs'][i]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Thi

In [275]:
temp[temp['songs'].apply(lambda x : len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date


In [276]:
# concat
val_10 = pd.concat([val_copy9,temp])
val_copy10 = val_10.drop_duplicates('id',keep = 'last')
val_copy10.reset_index(drop = True, inplace = True)

In [277]:
# 노래가 100개 아니면서 0개 이상인거, 즉 노래가 다 안채워진것
# 다채워짐
val_copy10[val_copy10['songs'].apply(lambda x : len(x) != 100 and len(x) > 0)]

,tags,id,plylst_title,songs,like_cnt,updt_date


#### 저장하기

In [278]:
val_copy10.to_json('datas/val_copy10.json')

#### 불러오기

In [279]:
val_copy10 = pd.read_json('datas/val_copy10.json')

In [280]:
# 태그가 10개 인거중에 이상한 애들이 있음.
val_copy10[val_copy10['tags'].apply(lambda x : len(x) == 10 and type(x) == str)]

,tags,id,plylst_title,songs,like_cnt,updt_date
22691,,138702,지겹게 들었다..2,[],20,2007-02-18 14:56:48.000
22706,up up up,68797,up up up!!,[],3,2006-11-29 10:14:56.000
22755,,5216,20160920 1,[],0,2016-09-20 18:39:36.000
22760,,45977,좋아요 너무 좋아요,[],3,2017-11-13 16:01:57.000
22763,,124962,테크토닉에 빠져들다,[],2397,2010-01-19 14:55:01.000
22786,,67776,#모두다몸을흔들어#,[],6,2015-02-14 23:05:31.000
22794,the noir,84271,THE NOIR .,[],1,2020-03-26 22:41:08.000
22807,i like it,131826,I Like it!,[],30,2014-03-30 10:06:57.000


In [281]:
# 그런애들 태그삭제
for i in val_copy10.index:
    if type(val_copy10.loc[i]['tags']) == str:
        val_copy10['tags'][i] = []
        val_copy10['tags'][i]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [282]:
# 태그가 10개가 아닌것 10개로 채우기
val_copy10[val_copy10['tags'].apply(lambda x : len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date
22691,[],138702,지겹게 들었다..2,[],20,2007-02-18 14:56:48.000
22706,[],68797,up up up!!,[],3,2006-11-29 10:14:56.000
22755,[],5216,20160920 1,[],0,2016-09-20 18:39:36.000
22760,[],45977,좋아요 너무 좋아요,[],3,2017-11-13 16:01:57.000
22763,[],124962,테크토닉에 빠져들다,[],2397,2010-01-19 14:55:01.000
...,...,...,...,...,...,...
23010,[],146744,,"[250811, 344947, 689313, 595345, 63785, 208011...",0,2015-10-02 14:50:11.000
23011,[],63799,,"[706946, 699047, 423412, 483887, 284971, 59787...",2,2014-09-28 13:46:15.000
23012,[],108712,,"[14483, 92664, 268608, 161546, 288855, 344398,...",1,2020-04-03 17:03:31.000
23013,[],86094,,"[101151, 552297, 652247, 322012, 659539, 25871...",14,2018-03-12 14:38:38.000


In [283]:
# 얘내를 10개로 채우자
temp = val_copy10[val_copy10['tags'].apply(lambda x: len(x) != 10)]
temp.reset_index(drop = True, inplace = True)

In [284]:
import random
random.seed(13)
tag_list = list(tag_songs['tags'])
for i in range(0, len(temp)):
    temp['tags'][i] = []
    s = list(map(str,random.sample(tag_list, 10)))
    temp['tags'][i] = list(set(s))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [285]:
# 10개 아닌거 확인
temp[temp['tags'].apply(lambda x: len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date


In [286]:
# concat
val_11 = pd.concat([val_copy10,temp])
val_copy11 = val_11.drop_duplicates('id',keep = 'last')
val_copy11.reset_index(drop = True, inplace = True)

In [287]:
# 태그10개아닌거 확인
val_copy11[val_copy11['tags'].apply(lambda x: len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date


#### 저장하기

In [288]:
val_copy11.to_json('datas/val_copy11.json')

#### 불러오기

In [289]:
val_copy11 = pd.read_json('datas/val_copy11.json')

In [290]:
val_copy11[val_copy11['tags'].apply(lambda x: len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date


In [291]:
# 송이 100개아닌거 확인
val_copy11[val_copy11['songs'].apply(lambda x: len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date
22814,"[특이한_음악_모음, 핫하다핫해, 잠잘때듣기좋은, 끈적끈적, 일렉뮤직, 녹는거, 이...",138702,지겹게 들었다..2,[],20,2007-02-18 14:56:48.000
22815,"[고향길, not캐롤but겨울노래, 음식점, 찌질감성, 가로수길, 김희성, 한여름추...",68797,up up up!!,[],3,2006-11-29 10:14:56.000
22816,"[기대고싶을때, 와인과어울리는, 동네, 하우스룰즈, 2018_2019, 93년생, ...",5216,20160920 1,[],0,2016-09-20 18:39:36.000
22817,"[토리켈리, 파티음악, 핑클의, 눈물을, 달달한팝송, 힙합명반, 가는해, 패셔너블,...",45977,좋아요 너무 좋아요,[],3,2017-11-13 16:01:57.000
22818,"[프로포즈, 비투비, supreme, 해나, 청승, 토끼, 맨아이트러스트, 노래선곡...",124962,테크토닉에 빠져들다,[],2397,2010-01-19 14:55:01.000
...,...,...,...,...,...,...
22917,"[별헤는밤, 그린플러그드경주, POPCCM, 훌라후프, 카밀라카베요, 힙합, 예언,...",72413,2+1-2=.....,[],9,2019-04-22 09:38:46.000
22918,"[마녀배달부키키, 여행드라이브, Mamasgun, 바에서듣는재즈, gryffin, ...",16538,이정도 일렉이면 딱좋아!,[],0,2015-10-09 19:25:45.000
22919,"[슈게이징, 국내락, 스노보드, 봄여울, 청사포끝집노래, 더셔를스, 평생, 광기, ...",18930,힙덕힙덕의 인디탐험기,[],1,2014-10-19 13:32:22.000
22920,"[DJ프리미어, 아무나, 국내락, 여름이, NOB4120304, 들어라, YMO, ...",43034,1111111111111111,[],0,2016-07-12 19:09:29.000


In [292]:
# 100개 채우기
temp = val_copy11[val_copy11['songs'].apply(lambda x: len(x) != 100)]
temp.reset_index(drop = True, inplace = True)

In [293]:
import random
random.seed(13)
song_list = list(song_tags['songs'])
for i in range(0, len(temp)):
    temp['songs'][i] = []
    s = list(map(str,random.sample(song_list, 100)))
    temp['songs'][i] = list(set(s))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [294]:
# concat
val_12 = pd.concat([val_copy11,temp])
val_copy12 = val_12.drop_duplicates('id',keep = 'last')
val_copy12.reset_index(drop = True, inplace = True)

In [295]:
# 확인
val_copy12[val_copy12['tags'].apply(lambda x: len(x) != 10)]

,tags,id,plylst_title,songs,like_cnt,updt_date


In [296]:
# 확인
val_copy12[val_copy12['songs'].apply(lambda x: len(x) != 100)]

,tags,id,plylst_title,songs,like_cnt,updt_date


In [297]:
# 태그 중복확인
no_tag = []
for i in val_copy12.index:
    if len(set(val_copy12['tags'][i])) != 10:
        no_tag.append(val_copy12.loc[i])

In [298]:
pd.DataFrame(no_tag)

,tags,id,plylst_title,songs,like_cnt,updt_date
21097,"[불후, 명곡, 전설, 노래, 최백호, 편, 곡, 및, 곡, 음원]",13508,불후의 명곡 전설을 노래하다(최백호편) 원곡 및 경연곡 음원,"[37748, 144663, 224921, 654757, 246531, 645489...",6,2016-02-16 10:44:18.000


In [299]:
val_copy12['tags'][21097] = list(set(val_copy12['tags'][21097])) + ['불후의명곡']
val_copy12.iloc[21097]['tags']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


['편', '명곡', '불후', '최백호', '전설', '노래', '곡', '음원', '및', '불후의명곡']

In [300]:
# 노래 중복확인
no_song = []
for i in val_copy12.index:
    if len(set(val_copy12['songs'][i])) != 100:
        no_tag.append(val_copy12.loc[i])

In [301]:
pd.DataFrame(no_song)

""


#### 저장하기

In [302]:
val_copy12.to_json('datas/val_copy12.json')

#### 불러오기

In [1]:
val_copy12 =pd.read_json('datas/val_copy12.json')

#### 제출용 저장

In [26]:
result = val_copy12[['id','songs','tags']]

In [27]:
results = []
for i in range(0,len(result)):
    results.append({'id' : int(result['id'][i]), 'songs' : list(map(int, result['songs'][i])), 'tags' : result['tags'][i]})

In [28]:
with open('datas/results.json', 'w', encoding='utf-8',) as make_file:json.dump(results, make_file, indent="\t",ensure_ascii=False)

#### 번외, songs가 0점 나와서 걍 랜덤하게 100개 다 꽂아넣는 식으로 해보자 - 해보니까, 결과저장할때 str 형식이면 안되내요

In [ ]:
# val_copy12 =pd.read_json('datas/val_copy12.json')

In [ ]:
# result = val_copy12[['id','songs','tags']]

In [10]:
# import random
# random.seed(13)
# song_list = list(song_tags['songs'])
# for i in range(0, len(result)):
#     result['songs'][i] = []
#     s = list(map(str,random.sample(song_list, 100)))
#     result['songs'][i] = list(set(s))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [24]:
# results = []
# for i in range(0,len(result)):
#     results.append({'id' : int(result['id'][i]), 'songs' : list(map(int, result['songs'][i])), 'tags' : result['tags'][i]})

In [25]:
# with open('datas/results.json', 'w', encoding='utf-8',) as make_file:json.dump(results, make_file, indent="\t",ensure_ascii=False)